#실행전 준비

데이터 출처: https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=580

github ipynb 출력 오류로 셀 출력 없는 버전이 업로드 되었습니다.

셀 출력 있는 버전은 colab에 업로드 되어있습니다.

링크: https://colab.research.google.com/drive/1RiVlmiTxmwd_ZvSnAjiOoMbk1u2Mf6M4?usp=sharing

In [ ]:
!pip install -q -U transformers bitsandbytes protobuf
!pip install -q -U peft gradio langchain_chroma langchain_huggingface langchain_community

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files

src = list(files.upload().values())[0]
open('summerizer.py','wb').write(src)

In [ ]:
from google.colab import files

src = list(files.upload().values())[0]
open('predictor.py','wb').write(src)

In [ ]:
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

#라이브러리 import

In [ ]:
import pickle
import gradio as gr
from summerizer import summarize_text
from predictor import predict_law

#test 데이터 로드

In [ ]:
with open("./final_test.pkl","rb") as fr:
    final_test = pickle.load(fr)

In [ ]:
type(final_test)

In [ ]:
final_test[21]

#프롬프트 제작 함수

In [ ]:
#gradio에서 받은 입력을 바탕으로 프롬프트 제작하는 함수
def make_prompt(court,case,detail_case,question, plaintiff, defendant, fact, conclusion):
    messages = {'messages':
                [
                    {'content': question, 'role': 'system'},
                    {'content': "사건유형 " + case + " 세부유형 " +  detail_case +  " " +  court +
                       " " +  plaintiff + " " + defendant + " " + fact
                       ,'role': 'user'},
                    {'content': conclusion,'role': 'assistant'}
                ]
               }
    return messages

#gradio 설정 및 실행

gradio 출력결과는 네이버 블로그를 참고해주세요.

랑크: https://blog.naver.com/gijong9806/224105514211

In [ ]:
with gr.Blocks() as demo:
    with gr.Column():
        with gr.Row():
            Dropdown1 = gr.Dropdown(["1심", "2심"], label =  "1심, 2심 선택", value="1심", interactive=True)
            Dropdown2 = gr.Dropdown(["민사", "형사", "행정"], label = "사건유형", value="민사", interactive=True)
            Dropdown3 = gr.Dropdown(["민사", "신청", "가사", "특허", "행정", "형사"], label ="세부유형", value="민사", interactive=True)
        with gr.Row():
            text1 = gr.Textbox(label="질문", lines=2)
        with gr.Row():
            text2 = gr.Textbox(label="원고", lines=5, interactive=True)
        with gr.Row():
            text3 = gr.Textbox(label="피고", lines=5, interactive=True)
        with gr.Row():
            text4 = gr.Textbox(label="기초사실", lines=5, interactive=True)
        with gr.Row():
            text5 = gr.Textbox(label="법원 판단", lines=5, interactive=True)

        with gr.Row():
            num = gr.Number(label="인덱스", interactive=True)
            example_btn = gr.Button("예시")
    with gr.Column():
        true_answer = gr.Textbox(label = "정답", lines=5)
        answer = gr.Textbox(label = "예측 결과", lines=5)
        rag = gr.Textbox(label = "참고자료", lines=5)
        answer_btn = gr.Button("변환")


    def example_prompt(n):
        idx = int(n)  # Number는 float로 들어오니까 안전하게 int 변환

        return (
            str(final_test[idx]['messages'][1]['content'][16:18]),
            str(final_test[idx]['messages'][1]['content'][5:7]),
            str(final_test[idx]['messages'][1]['content'][13:15]),
            final_test[idx]['messages'][0]['content'],
            "원고 측의 주장 " + final_test[idx]['messages'][1]['content'].split('피고 측의 주장')[0].split('원고 측의 주장')[1],
            "피고 측의 주장 " + final_test[idx]['messages'][1]['content'].split('피고 측의 주장')[1].split('기초사실')[0],
            "기초사실 " + final_test[idx]['messages'][1]['content'].split('기초사실')[1],
            final_test[idx]['messages'][2]['content'],
        )

    def run_predict(drop1, drop2, drop3, q, plaintiff, defendant, facts, court_judgment):
        """
        버튼 클릭 시 실행되는 함수.
        - 원고/피고/기초사실을 요약한 뒤
        - predict_law 클래스로 추론 실행
        """
        # 빈 문자열 방지용 처리(없어도 상관없긴함)
        plaintiff = plaintiff or ""
        defendant = defendant or ""
        facts = facts or ""
        court_judgment = court_judgment or ""

        # summarize_text 수행 (Summerizer의 summarize_text 사용)
        plaintiff_sum = summarize_text(plaintiff) if plaintiff.strip() else ""
        defendant_sum = summarize_text(defendant) if defendant.strip() else ""
        facts_sum = summarize_text(facts, 400) if facts.strip() else ""

        # predict_law에서 기대하는 데이터 포맷에 맞추기
        data = [
            drop1,           # 0: 1심/2심
            drop2,           # 1: 사건유형
            drop3,           # 2: 세부유형
            q,               # 3: 질문/시스템 프롬프트
            plaintiff_sum,   # 4: 원고 주장 (요약)
            defendant_sum,   # 5: 피고 주장 (요약)
            facts_sum,       # 6: 기초사실 (요약)
            court_judgment,  # 7: 법원 판단 (정답/참고용)
        ]

        predictor = predict_law(data)
        return predictor.prompt['messages'][2]['content'], predictor.answer, predictor.rag  # Gradio의 output으로 전달

    example_btn.click(#버튼 클릭시 test 데이터 샘플 출력
        fn=example_prompt,
        inputs=num,
        outputs=[Dropdown1,Dropdown2,Dropdown3,text1, text2, text3, text4, text5],
    ),
    answer_btn.click(#버튼 클릭시 gradio ui로 입력받은 데이터를 run_predict 함수로 넘겨서 모델 예측 실행
        fn=run_predict,
        inputs=[Dropdown1,Dropdown2,Dropdown3,text1, text2, text3, text4, text5],
        outputs=[true_answer, answer,rag]
    )

demo.launch()
